In [273]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import category_encoders as ce
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import RFE

plt.style.use('seaborn-dark-palette')

warnings.filterwarnings(action='ignore', category=DataConversionWarning)
pd.options.display.max_columns = 200


In [274]:
train_features = pd.read_csv('Downloads/train_features.csv')
test_features = pd.read_csv('Downloads/test_features.csv')
train_labels = pd.read_csv('Downloads/train_labels.csv')
test_labels = pd.read_csv('Downloads/SubmissionFormat.csv')

In [695]:
train_features = pd.read_csv('Downloads/4910797b-ee55-40a7-8668-10efd5c1b960.csv')
test_features = pd.read_csv('Downloads/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
train_labels = pd.read_csv('Downloads/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
test_labels = pd.read_csv('Downloads/SubmissionFormat.csv')
df = train_features.append(test_features)

In [696]:
df['month_recorded'] = pd.to_datetime(df.date_recorded).dt.month
df['year_recorded'] = pd.to_datetime(df.date_recorded).dt.year
df['dayofwk_recorded'] = pd.to_datetime(df.date_recorded).dt.weekday
df['construction_year_missing'] = (df.construction_year == 0)
df['constructYear_is_RecordYear'] = (df.year_recorded == df.construction_year)
df['construction_year_from2014'] = 2014 - df.construction_year
df.construction_year = df.construction_year.replace({0:1986})
df['is_weekend?'] = ((df.dayofwk_recorded == 6) | (df.dayofwk_recorded == 0))
df['construction_year_from_record'] = df.year_recorded - df.construction_year

df['gps_height_zero'] = (df.gps_height == 0)
df['gps_height_negative'] = (df.gps_height == 0)
df['amount_tsh_zero'] = (df.amount_tsh == 0)

df['source_marchine_dbh?'] = (df.source == 'machine dbh')
df['scheme_name_missing?'] = (df.scheme_name != 'missing')
df['ward_is_Igosi?'] = (df.ward == 'Igosi')

df.public_meeting = df.public_meeting.fillna('missing')

df['mgmt_is_wua?'] = (df.management == 'wua')
df['is_parastatal'] = (df.management_group == 'parastatal')
df['region_code_11?'] = (df.region_code == 11)
df['region_code_2?'] = (df.region_code == 2)

df['my_source_classes'] = df.source_type.map(
    {
    'spring': 'good', 'shallow well': 'ok',
    'borehole':'ok', 'river/lake':'good',
    'rainwater harvesting': 'good', 'dam': 'bad',
    'other': 'ok'})

df.funder = df.funder.fillna('missing')
df.installer = df.installer.fillna('missing')
df.public_meeting = df.public_meeting.fillna('missing')


df['yearMo_reacorded_fractional'] = df.year_recorded + (df.month_recorded -1)/ 12
df['year_recorded_from_2014'] = 2014 -- df.yearMo_reacorded_fractional

df.installer = df.installer.astype(str)

df.funder = df.funder.str.lower()
df.installer = df.installer.str.lower()
df['funder_is_installer'] = (df.funder == df.installer)


df['population_over_215'] = (df.population >= 215)

df.installer.astype(str)

df['funder_is_gov'] = df.funder.str.contains('gov')
df['installer_is_gov'] = df.installer.str.contains('gov')
df['funder_is_wor'] = df.funder.str.contains('wor')
df['more_than_one_basin'] = (df.basin.str.contains('/'))
df['more_than_one_funder'] = df.funder.str.contains('/')
df['space_in_funder_name'] = df.funder.str.contains(' ')
df['space_in_installer_name'] = df.installer.str.contains(' ')

df.lga = df.lga.str.lower()
df.ward = df.ward.str.lower()

df['payment_unknown_doesnt_pay'] = ((df.payment == 'unknown') | (df.payment == 'never pay'))

df.scheme_name = df.scheme_name.str.lower()
df['scheme_name_len_1'] = (df.scheme_name.str.len() == 1)
df['scheme_name_con_gov'] = (df.scheme_name.str.contains('gov'))
df['scheme_name_con_supply'] = (df.scheme_name.str.contains('supply'))
df['scheme_name_con_space'] = (df.scheme_name.str.contains(' '))

df.scheme_management = df.scheme_management.str.lower()
df['scheme_mgmt_contains_w'] = (df.scheme_management.str.contains('w'))
df['scheme_mgmt_contains_p'] = (df.scheme_management.str.contains('p'))
df['scheme_mgmt_is_not_known'] = (df.scheme_management.str.contains('other') | df.scheme_management.str.contains('none') | df.scheme_management.str.contains('missing'))

df['num_private_over0'] = (df.num_private > 0)
df['num_private_over1'] = (df.num_private > 1)
df['num_private_over10'] = (df.num_private > 10)
df['num_private_over100'] = (df.num_private > 100)

# june, july, august, september, october is tanzania dry season
df['quantity_is_seasonal'] = (df.quantity == 'seasonal')
df['dry_season'] = ((df.month_recorded >= 6) & (df.month_recorded <= 10))
df['wet_season'] = ((df.month_recorded == 12) | (df.month_recorded <= 4))

df['day_of_year'] = pd.to_datetime(df.date_recorded).dt.dayofyear
df['days_away_from_middle_of_year'] = 180 - df.day_of_year

df.wpt_name = df.wpt_name.str.lower()
df['wpt_name_none'] = (df.wpt_name == 'none')

df.region_code = df.region_code.astype(str)
df.district_code = df.district_code.astype(str)

df['gps_height_squared'] = df.gps_height ** 2
df['amount_tsh_squared'] = df.amount_tsh ** 2
df['days_away_from_middle_of_year_squared'] = df.days_away_from_middle_of_year ** 2
df['dayofwk_recorded_str'] = df.dayofwk_recorded.astype(str)
df['construction_year_from2014_squared'] = df.construction_year_from2014 ** 2

df['source_discrep'] = (df.source != df.source_type)
df['quality_discrep'] = (df.water_quality != df.quality_group)

df['extraction_type_includes_other'] = df.extraction_type.str.contains('other')
df['extraction_type_gravity'] = (df.extraction_type_class == 'gravity')


installer_names = list(df.installer.value_counts().index)
installer_instances = list(df.installer.value_counts().values)

installer_map = {}

for installer, instances in zip(installer_names, installer_instances):
    installer_map[installer] = instances


funder_names = list(df.funder.value_counts().index)
funder_instances = list(df.installer.value_counts().values)

funder_map = {}

for funder, instances in zip(funder_names, funder_instances):
    funder_map[funder] = instances
    
df['installer_experience'] = df.installer.map(installer_map)
df['funder_experience'] = df.funder.map(funder_map)



installer_names2 = list(df.groupby('installer').construction_year.min().index)
installer_min_year = list(df.groupby('installer').construction_year.min().values)

installer_map2 = {}

for installer, yr in zip(installer_names2, installer_min_year):
    installer_map2[installer] = yr
    

df['installer_first_construct_year'] = df.installer.map(installer_map2)

installer_names3 = list(df.groupby('installer').construction_year.max().index)
installer_max_year = list(df.groupby('installer').construction_year.max().values)

installer_map3 = {}

for installer, yr in zip(installer_names3, installer_max_year):
    installer_map3[installer] = yr
    

df['installer_last_construct_year'] = df.installer.map(installer_map3)


installer_names4 = list(df.groupby('installer').construction_year.mean().index)
installer_mean_year = list(df.groupby('installer').construction_year.mean().values)

installer_map4 = {}

for installer, yr in zip(installer_names4, installer_mean_year):
    installer_map4[installer] = yr
    

df['installer_mean_construction_year'] = df.installer.map(installer_map4)

df['installer_funder_exp'] = df.installer_experience + df.funder_experience
df['record_year_from_2014'] = 2014 - df.year_recorded

loc_features = ['gps_height', 'longitude', 'latitude']

std_sc = StandardScaler()
df_std = std_sc.fit_transform(df[loc_features])
pca_all = PCA(3)
pc_all = pca_all.fit_transform(X=df_std)

pca3_df = pd.DataFrame({'pc1':pc_all[:,0],'pc2':pc_all[:,1], 'pc3': pc_all[:, 2]})


kms = [2, 5, 15]

for means in kms:
    km = KMeans(n_clusters=means)
    km = km.fit(pca3_df[['pc1', 'pc2', 'pc3']])
    df['XYZ_Cluster_Label_' + str(means)] = km.labels_

df['installer_longevity'] = df.installer_last_construct_year - df.installer_first_construct_year
df['installer_installations_per_year'] = df.installer_experience / df.installer_longevity

for name in list(df.wpt_name.value_counts()[1:15].index):
    df['wpt_name_' + name] = (df.wpt_name == name)

df.subvillage = df.subvillage.str.lower()
for village in list(df.subvillage.value_counts()[:10].index):
    df['subvillage_' + village] = (df.subvillage == village)
    
for lg in list(df.lga.value_counts()[:19].index):
    df['lga_' + lg] = (df.lga == lg)
    
for wrd in list(df.ward.value_counts()[:13].index):
    df['ward_' + wrd] = (df.ward == wrd)
    
for fndr in list(df.funder.value_counts()[:12].index):
    df['funder_' + fndr] = (df.funder == fndr)
    
for inst in list(df.installer.value_counts()[:9].index):
    df['installer_' + inst] = (df.installer == inst)
    
df['lga_ward_same'] = (df.lga == df.ward)

df = df.fillna('missing')

for ins in list(df.installer.value_counts()[:9].index):
    df['installer_' + ins] == (df.installer == ins)

df.payment_type = pd.factorize(df.payment_type)[0]
df.water_quality = pd.factorize(df.water_quality)[0]
df.scheme_management = pd.factorize(df.scheme_management)[0]
df.extraction_type = pd.factorize(df.extraction_type)[0]
df.funder = pd.factorize(df.funder)[0]
df.waterpoint_type = pd.factorize(df.waterpoint_type)[0]
df.lga = pd.factorize(df.lga)[0]
df.source = pd.factorize(df.source)[0]
df.quantity = pd.factorize(df.quantity)[0]
df.basin = pd.factorize(df.basin)[0]
df.management = pd.factorize(df.management)[0]
df.region = pd.factorize(df.region)[0]
df.district_code = pd.factorize(df.district_code)[0]
df.my_source_classes = pd.factorize(df.my_source_classes)[0]
df.installer = pd.factorize(df.installer)[0]


df['amount_tsh_not0_over500'] = ((df.amount_tsh != 0) & (df.amount_tsh >= 500))
df['amount_tsh_not0_over1000'] = ((df.amount_tsh != 0) & (df.amount_tsh >= 1000))
df['amount_tsh_not0_over5000'] = ((df.amount_tsh != 0) & (df.amount_tsh >= 10000))
df['amount_tsh_not0_over10000'] = ((df.amount_tsh != 0) & (df.amount_tsh >= 10000))

df['population_over1'] = (df.population >= 2)
df['population_under50'] = (df.population < 50)

df['gps_height_pct'] = df.gps_height / 2777

df['region_district_combo'] = df.region.astype(str) + df.district_code.astype(str)
df['water_region_district_combo'] = df.waterpoint_type.astype(str) + df.region_district_combo
df['water_region_district_factorized'] = pd.factorize(df.water_region_district_combo)[0]
df['region_district_combo_factorized'] = pd.factorize(df.region_district_combo)[0]

df['basin_region_district_combo'] = df.region_district_combo + df.basin.astype(str)
df['basin_region_district_factorized'] = pd.factorize(df.basin_region_district_combo)[0]

df['management_group_factorized'] = pd.factorize(df.management_group)[0]
df['mgmt_mgmt_group_combined'] = df.management.astype(str) + df.management_group_factorized.astype(str)
df['mgmt_mgmt_group_combined'] = pd.factorize(df.mgmt_mgmt_group_combined)[0]

def mac_wpt_name_mapper(wpt_name):
    
    # 75 wpt names
    
    wpt_names = ['none', 'shuleni', 'zahanati', 'msikitini', 'kanisani', 'sokoni',
       'bombani', 'ofisini', 'school', 'shule ya msingi', 'shule', 'sekondari',
       'muungano', 'mkombozi', 'upendo', 'madukani', 'mbugani',
       'kituo cha afya', 'umoja', 'mkuyuni', 'hospital', 'center', 'kisimani',
       'ccm', 'mtakuja', 'ofisi ya kijiji', 'songambele', 'bwawani', 'tankini',
       'bondeni', 'maendeleo', 'kilabuni', 'mbuyuni', 'uwanjani', 'kijiweni',
       'shuleni sekondari', 'miembeni', 'mnadani', 'secondary', 'rc church',
       'mwembeni', 'amani', 'majengo', 'dispensary', 'mahakamani', 'mashineni',
       'tank la shule', 'church', 'tenkini', 'mtoni', 'darajani', 'barabarani',
       'kwa juma', 'kwa john', 'tumaini', 'polisi', 'tupendane', 'mkwajuni',
       'tangini', 'kwa joseph', 'mission', 'kwa charles', 'ushirika',
       'magereza', 'kwa', 'jamii', 'mshikamano', 'mlimani', 'joshoni',
       'stendi', 'juhudi', 'kwa samweli', 'ofisi ya kata', 'kolongoni',
       'machinjioni']
    
    if wpt_name in wpt_names:
        return wpt_name
    
    return 'misc'

df['mac_wpt_name'] = df.wpt_name.apply(mac_wpt_name_mapper)
df['mac_wpt_name'] = pd.factorize(df.mac_wpt_name)[0]


def mac_installer_mapper(installer):
    
    # 75 installer names
    
    installers =  [5,  19,  28,  35,  31,  23,   9,  16,  30,  60,  21,  79,   2,
             12,  48,  37, 142,  73,  58,  25,  11,  62,  78,  93,  43,  92,
             18,  36,  33, 162, 186,   3,  34,  97,  66,  87, 232,  95,  39,
            130,  80,  67, 205,   6,  10, 132, 197, 195,  85,  27, 156,  69,
              8, 165,  91, 179,  84, 160, 115, 123,   7, 143,  90,  81,  14,
              4,  44,  59, 135, 121,  40, 149,  70,  13,  26, 145, 184, 146,
            173, 246,  64, 181,  71, 223, 252,   0,  22, 227, 300,  54,  46,
            321, 347, 139, 169, 102, 236, 310, 141, 137]
    if installer in installers:
        return installer
    
    return 'misc'
df['mac_installer'] = df.installer.apply(mac_installer_mapper)
df['mac_installer'] = pd.factorize(df.mac_installer)[0]

def mac_funder_mapper(funder):
    
    # 20 funder names
    
    funders = [20,  25,  11,  16,   7,  68,  39,  12,   3,  63,  81,  32,  23,
              6,  70,  41,  58,  22,  56,  89,  45,  59, 132,  90,  31,  88,
             73, 114,  46,   8,  15,  18,  10, 165, 202,  98,   0,  42,  29,
             53, 142,  72, 210, 112, 226, 251, 180, 128,  13, 152,  75, 163,
            129, 117, 101,  37,  35,  80,  62,  83, 130,   5,  64, 149, 189,
            102, 100, 170,  49, 159, 213, 123,  51,  82,  84, 262, 135, 138,
             40, 107, 245, 109, 253, 146,  33, 167,  97, 166,  94,  76,  69,
            240,  17, 252,  55,  50, 343,  24, 282, 162]
    
    if funder in funders:
        return funder
    
    return 'misc'

df['mac_funder'] = df.funder.apply(mac_funder_mapper)
df['mac_funder'] = pd.factorize(df.mac_funder)[0]

def mac_subvillage_mapper(subvillage):
    
    # 100 subvillage names
    
    subvillages = ['shuleni', 'majengo', 'madukani', 'missing', 'kati', 'mtakuja',
       'sokoni', 'm', 'muungano', 'mbuyuni', 'songambele', 'mlimani',
       'miembeni', 'msikitini', '1', 'kanisani', 'kibaoni', 'mjini',
       'mjimwema', 'mapinduzi', 'mwenge', 'mkwajuni', 'i', 'bondeni', 'azimio',
       'mabatini', 'amani', 'mbugani', 'mission', 'kichangani',
       'mtaa wa kitunda kati', 'changombe', 'senta', 'bwawani', 'zahanati',
       'nyerere', 'misufini', 'kisiwani', 'center', 'k', 'kawawa', 'maendeleo',
       'mtaa wa kivule', 'ccm', 'ikulu', 'ushirika', 'barabarani', 'gulioni',
       'shule', 'msufini', 'ilala', 'magharibi', 'msumbiji', 'darajani',
       'marurani juu', 'uzunguni', 'marurani kati', 'dodoma', 'misheni',
       'mpakani', 'zaire', 'ofisini', 'elimu', 'sokoine', 'kilimahewa',
       'bomani', 'magomeni', 'kaloleni', 'ngelele', 'minazini', 'mkuyuni',
       'mnadani', 'ujamaa', 'katumba', 'vikuge', 'muongozo', 'mnazi mmoja',
       'usalama', 'mashariki', 'mikoroshini', 'migombani', 'mwangaza',
       'kijijini', 'mwabasabi', 'bombani', 'kiwanjani', 'mnazimmoja',
       'chemchem', 'mchangani', 'uswahilini', 'mtaa wa vikongoro', 'mtoni',
       'l', 'njiapanda', 'stendi', 'gezaulole', 'isanga', 'njia panda',
       'kakola', 'tandika']
    
    if village in subvillages:
        return village
    
    return 'misc'

df['mac_subvillage'] = df.subvillage.apply(mac_subvillage_mapper)
df['mac_subvillage'] = pd.factorize(df.mac_subvillage)[0]

def mac_ward_mapper(ward):
    
    # 100 ward names
    
    wards = ['igosi', 'imalinyi', 'siha kati', 'mdandu', 'nduruma', 'kitunda',
       'mishamo', 'msindo', 'chalinze', 'maji ya chai', 'usuka',
       'ngarenanyuki', 'chanika', 'vikindu', 'mtwango', 'zinga/ikerege',
       'matola', 'magomeni', 'maramba', 'mvomero', 'wangingombe', 'ifakara',
       'itete', 'olkokola', 'kikatiti', 'nkoma', 'maposeni', 'mahongole',
       'rujewa', 'igongolo', 'masama magharibi', 'mlangali', 'hedaru',
       'kidatu', 'simbo', 'ihanda', 'lupalilo', 'kagongo', 'kiranyi',
       'nkungulu', 'isongole', 'yombo', 'chinamili', 'makwale', 'kanga',
       'diongoya', 'soga', 'kimochi', 'siha mashariki', 'masama mashariki',
       'tinde', 'ilolangulu', 'malindi', 'makuyuni', 'bugarama',
       'mabwerebwere', 'mkongo', 'kibaoni', 'mangula', 'ruanda', 'muzye',
       'wino', 'lembeni', 'mhunze', 'kenyamonta', 'mahembe',
       'kirua vunjo kusini', 'mamire', 'kakonko', 'rusumo', 'mkula',
       'chanzuru', 'mlali', 'leguruki', 'ubaruku', 'murufiti', 'bungu',
       'kisesa', 'kibosho magharibi', 'magagura', 'matendo', 'lumemo',
       'kifanya', 'chimala', 'mlangarini', 'uwemba', 'mpuguso', 'ruhembe',
       'munyegera', 'ipande', 'rungwe mpya', 'muhinda', 'igurusi',
       'mbulumbulu', 'ilkidinga', 'msia', 'lupembe', 'busole', 'bumera',
       'kidodi']
    if ward in wards:
        return ward
    
    return 'misc'

df['mac_wards'] = df.ward.apply(mac_ward_mapper)
df['mac_wards'] = pd.factorize(df.mac_wards)[0]


df['cartesian_lat'] = np.cos(df.latitude) * np.cos(df.longitude) * df.gps_height
df['cartesian_long'] = np.cos(df.latitude) * np.sin(df.longitude) * df.gps_height
df['cartesian_altitude'] = np.sin(df.latitude) * df.gps_height




df['construction_year_over2002'] = (df.construction_year > 2002) # openly said they wanna be better
df['construction_year_over2006'] = (df.construction_year > 2006) # legislation
df['construction_year_over2009'] = (df.construction_year > 2009) # new reporting standards

df['basin_is_3'] = (df.basin == 3)


df['mac_population_encoded'] = df.population.map({1:1, 0:2}).fillna(3)
df['wpt_name_school'] =((df.wpt_name.str.contains('shul')) | (df.wpt_name.str.contains('school')))
df['wpt_name_kwa'] =((df.wpt_name.str.contains('kwa')))
df['wpt_name_church'] = ((df.wpt_name.str.contains('msikitini')) | (df.wpt_name.str.contains('kanisani')) | (df.wpt_name.str.contains('kwa')))
df['wpt_name_charles'] = (df.wpt_name.str.contains('charles'))
df['wpt_name_market'] = ((df.wpt_name.str.contains('soko')) | (df.wpt_name.str.contains('madukani')))
df['wpt_name_hospital'] = ((df.wpt_name.str.contains('kituo')) | (df.wpt_name.str.contains('afya')))

In [697]:
from scipy.spatial import ConvexHull


def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371000):
    """
    Vectorized great circle distance between two points
    (lat, lon) specified in decimal degrees or in radians)
    Returns units in meters
    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


def append_dist_to_group_center(df, group_col='basin'):
    lats = {}
    lons = {}

    grouped = df.groupby(group_col)
    for group_name, group_df in grouped:
        points = list(zip(group_df.latitude, group_df.longitude))

        hull = ConvexHull(points)
        form = [points[i] for i in hull.vertices]

        lats[group_name] = np.mean(hull.points[hull.vertices,0])
        lons[group_name] = np.mean(hull.points[hull.vertices,1])

    centroids_df = (pd.DataFrame([lats, lons]).T
                      .rename(columns={0:f'{group_col}_lat', 1:f'{group_col}_lon'}))

    df = df.merge(centroids_df, left_on=group_col, right_index=True)

    df[f'dist_to_{group_col}'] = haversine(df.latitude, df.longitude,
                                           df[f'{group_col}_lat'], df[f'{group_col}_lon'])
    return df




In [698]:
df = append_dist_to_group_center(df)

In [699]:
train = pd.merge(train_labels, df, on='id', how='inner')
train['functional'] = (train.status_group == 'functional')

In [700]:
def macs_labelEncode_meaningful(df, train, cols, target):
    
    for col in cols:
        
        cur_encodes = pd.factorize(train.pivot_table(index=col, values=target).sort_values(target).index)[1].tolist()
        fnl_encodes = pd.factorize(train.pivot_table(index=col, values=target).sort_values(target).index)[0].tolist()
        
        mapper_dic = dict(zip(cur_encodes, fnl_encodes))
        
        df['MeaningfulEnc_' + col] = df[col].map(mapper_dic)
        
    return df

make_me_meaningful = ['basin', 'region', 'lga', 'payment_type',
                      'quantity', 'source', 'waterpoint_type',
                      'district_code', 'mac_installer', 'mac_funder',
                      'scheme_management', 'XYZ_Cluster_Label_15']


df = macs_labelEncode_meaningful(df, train, make_me_meaningful, 'functional')

In [701]:
train = pd.merge(train_labels, df, on='id', how='inner')
test = pd.merge(test_labels, df, on='id', how='inner')

In [702]:
df['mac_population_encoded'] = df.population.map({1:1, 0:2}).fillna(3)
df['wpt_name_school'] =((df.wpt_name.str.contains('shul')) | (df.wpt_name.str.contains('school')))
df['wpt_name_kwa'] =((df.wpt_name.str.contains('kwa')))
df['wpt_name_church'] = ((df.wpt_name.str.contains('msikitini')) | (df.wpt_name.str.contains('kanisani')) | (df.wpt_name.str.contains('kwa')))
df['wpt_name_charles'] = (df.wpt_name.str.contains('charles'))


train['mac_population_encoded'] = train.population.map({1:1, 0:2}).fillna(3)

In [703]:
train['functional'] = (train.status_group == 'functional')
df['wpt_name_charles'] = (df.wpt_name.str.contains('charles'))

In [571]:
train[train.wpt_name_kwa].shape

(23423, 239)

In [568]:
df.mac_population_encoded

0        3.0
11       3.0
19       1.0
20       3.0
33       3.0
36       2.0
41       2.0
47       2.0
52       2.0
80       2.0
109      2.0
113      2.0
120      2.0
124      2.0
135      3.0
140      2.0
211      2.0
213      2.0
215      2.0
254      2.0
270      2.0
275      3.0
303      2.0
307      2.0
312      2.0
328      3.0
332      2.0
348      3.0
362      2.0
373      3.0
        ... 
13676    2.0
13709    2.0
13723    3.0
13770    2.0
13818    2.0
13845    2.0
13847    2.0
13887    3.0
13893    1.0
13908    3.0
14026    3.0
14043    2.0
14058    1.0
14085    2.0
14089    3.0
14125    2.0
14169    2.0
14174    3.0
14182    2.0
14216    1.0
14231    3.0
14257    2.0
14260    2.0
14270    3.0
14290    2.0
14291    2.0
14294    3.0
14295    1.0
14296    3.0
14301    2.0
Name: mac_population_encoded, Length: 73758, dtype: float64

In [708]:
df['best_wards'] = ((df.ward == 'igosi') | (df.ward == 'imalinyi') | (df.ward == 'siha kati'))
df['second_best_wards'] = ((df.ward == 'mdandu') | (df.ward == 'nduruma'))
df.public_meeting = pd.factorize(df.public_meeting)[0]

train['best_wards'] = ((train.ward == 'igosi') | (train.ward == 'imalinyi') | (train.ward == 'siha kati'))
train['second_best_wards'] = ((train.ward == 'mdandu') | (train.ward == 'nduruma'))

In [710]:
df.public_meeting = pd.factorize(df.public_meeting)[0]
train = pd.merge(train_labels, df, on='id', how='inner')
test = pd.merge(test_labels, df, on='id', how='inner')


In [713]:
new_features = ['latitude', 'longitude', 'gps_height',
                'mac_funder', 'mac_installer', 'scheme_management', 'extraction_type', 'management', 
                'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type',
                'basin', 'region', 'lga', 'district_code', 'population', 'amount_tsh',
                'construction_year_from_record', 'day_of_year', 'funder_is_gov', 'my_source_classes',
                'best_wards', 'installer_experience', 'mac_population_encoded', 'funder_experience', 
                'second_best_wards', 'num_private_over100', 'public_meeting', 'mac_wards', 'amount_tsh_not0_over1000' ]

len(new_features)

32

In [714]:
X = train[new_features]
y = train.status_group

#pipe = make_pipeline(RobustScaler(), XGBClassifier(n_estimators=80, objective = 'multi:softprob', booster = 'gbtree', nrounds = 'min.error.idx', 
#                    num_class = 4, maximize = False, eval_metric = 'merror', eta = .3,
#                     max_depth = 16, colsample_bytree = 1, reg_lambda=5))

pipe = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', tree_method='exact',
                      num_class = 4, maximize = False, eval_metric = 'merror', eta = .1,
                      max_depth = 14, colsample_bytree=.35)


scores = cross_validate(pipe, X, y, scoring='accuracy', cv=3,verbose=10, return_train_score=True)

[CV]  ................................................................


C:\Users\mac\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mac\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.7s remaining:    0.0s


[CV] ....................... , score=0.8134343434343434, total=  39.2s
[CV]  ................................................................


C:\Users\mac\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mac\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


[CV] ....................... , score=0.8134848484848485, total=  35.5s
[CV]  ................................................................


KeyboardInterrupt: 

In [612]:
print(scores['test_score'].mean())
scores

0.8103367003367002


{'fit_time': array([30.82181549, 35.66789603, 30.86454654]),
 'score_time': array([1.83010626, 1.83110118, 1.88592386]),
 'test_score': array([0.81075758, 0.81136364, 0.80888889]),
 'train_score': array([0.93406566, 0.93487374, 0.93659091])}

In [613]:
pipe.fit(X,y)

for feat, imp in zip(new_features, pipe.feature_importances_):
    print(feat, imp)

latitude 0.012750058
longitude 0.011068806
gps_height 0.011133743
mac_funder 0.013615662
mac_installer 0.013209927
scheme_management 0.01716719
extraction_type 0.048680663
management 0.015412217
payment_type 0.02960061
water_quality 0.021627367
quantity 0.27013025
source 0.022972358
waterpoint_type 0.090314835
basin 0.018141368
region 0.017756391
lga 0.02238242
district_code 0.017273735
population 0.012270914
amount_tsh 0.02065708
construction_year_from_record 0.016191898
day_of_year 0.012359123
funder_is_gov 0.023413325
my_source_classes 0.036156904
best_wards 0.021932336
installer_experience 0.01551933
mac_population_encoded 0.017334022
funder_experience 0.013907895
mac_wards 0.018651767
gps_height_squared 0.010157934
public_meeting 0.014457481
num_private_over100 0.003974749
second_best_wards 0.007880439
wpt_name_kwa 0.010896024
amount_tsh_not0_over1000 0.018854113


In [621]:
test = pd.merge(test_labels, df, on='id', how='inner')

In [625]:
train['all_functional'] = ((train.status_group == 'functional') | (train.status_group == 'functional needs repair'))
train['all_needs_repair'] = ((train.status_group == 'non functional') | (train.status_group == 'functional needs repair'))

pipe = ensemble.RandomForestClassifier(500, min_samples_leaf=5)


X_test = test[new_features]

X = train[new_features]
y_func = train.all_functional
y_nonfunc = train.all_needs_repair

func_probs = pipe.fit(X, y_func).predict_proba(X_test)
nonfunc_probs = pipe.fit(X, y_nonfunc).predict_proba(X_test)

In [633]:
bayes = pd.DataFrame({'id': test.id, 'non functional': nonfunc_probs.T[1], 'functional': func_probs.T[1]})
bayes['functional_needs_repair'] = bayes['non functional'] * bayes['functional']

In [656]:
bayes['not_normal'] = bayes['non functional'] + bayes.functional + bayes.functional_needs_repair 

In [658]:
for col in ['non functional', 'functional', 'functional_needs_repair']:
    bayes[col] = bayes[col] / bayes.not_normal

In [666]:
bayes['guess_functional_needs_repair'] = (bayes.not_normal > bayes.not_normal.quantile(.97))

In [670]:
bayes['status_group'] = bayes.guess_functional_needs_repair.replace({True: 'functional_needs_repair'})

In [673]:
bayes['nonfunc'] = ((bayes['non functional'] > bayes['functional']) & (bayes.status_group == False))

In [678]:
bayes.nonfunc = bayes.nonfunc.replace({True: 'non functional'})

,id,non functional,functional,functional_needs_repair,status_group,not_normal,guess_functional_needs_repair,nonfunc,fnl_functional
0,50785,0.382835,0.405285,0.211880,False,1.365582,False,False,True
1,51630,0.219636,0.583643,0.196721,False,1.534612,False,False,True
2,17168,0.222971,0.594983,0.182046,False,1.372231,False,False,True
3,45559,0.993885,0.003062,0.003053,False,1.003390,False,non functional,False
4,49871,0.024236,0.951891,0.023873,False,1.034832,False,False,True


In [685]:
bayes['fnl_functional'] = ((bayes.status_group == False) & (bayes.nonfunc == False))

In [692]:
bayes.fnl_functional = bayes.fnl_functional.replace({True: 'functional'})

In [694]:
bayes.shape

(14358, 9)

In [693]:
bayes.to_csv('waterpipe_bayes.csv')

In [217]:
df['wpt_name_school'] =((df.wpt_name.str.contains('shul')) | (df.wpt_name.str.contains('school')))
df['wpt_name_kwa'] =((df.wpt_name.str.contains('kwa')))
df['wpt_name_church'] = ((df.wpt_name.str.contains('msikitini')) | (df.wpt_name.str.contains('kanisani')) | (df.wpt_name.str.contains('kwa')))
df['wpt_name_charles'] = (df.wpt_name.str.contains('msikitini'))
df['wpt_name_market'] = ((df.wpt_name.str.contains('soko')) | (df.wpt_name.str.contains('madukani')))
df['wpt_name_hospital'] = ((df.wpt_name.str.contains('kituo')) | (df.wpt_name.str.contains('afya')))

In [271]:
train[train.wpt_name.str.contains('afya')].functional.mean()

0.5906040268456376

In [717]:
pipe = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', tree_method='exact',
                      num_class = 4, maximize = False, eval_metric = 'merror', eta = .1,
                      max_depth = 14, colsample_bytree=.35)

X = train[new_features]
y = train.status_group

X_test = test[new_features]

pd.DataFrame({'id': test.id, 'status_group': pipe.fit(X, y).predict(X_test)}).to_csv('waterpipes_pumpitup3.csv')

C:\Users\mac\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [192]:
def mac_waterpoint_name2(wpt_name):
    
    # wpt_names dog
    
    schools = ['school', 'shul']
    
    if wpt_name in schools:
        return 'mac_school'
    
    offices = ['ofisi']
    if wpt_name in offices:
        return 'mac_offices'
    
    churches = ['msikitini', 'kanisani', 'kwa']
    if wpt_name in churches:
        return 'mac_churches'
    
    hospitals = ['kituo']
    if wpt_name in churches:
        return 'hospitals'
    
    return 'misc'

df['mac_waterpoint_name2'] = df.wpt_name.apply(mac_waterpoint_name2)

In [202]:
df['mac_waterpoint_name2'] = df.wpt_name.apply(mac_waterpoint_name2)

In [203]:
df.mac_waterpoint_name2.value_counts()

misc            72422
mac_churches     1075
mac_school        258
mac_offices         3
Name: mac_waterpoint_name2, dtype: int64

In [ ]:
df.head()

In [ ]:
df.head()

In [370]:
df.public_meeting.value_counts()

True       63319
False       6320
missing     4119
Name: public_meeting, dtype: int64

In [608]:
train.pivot_table(index='wpt_name_church', values='functional')

KeyError: 'functional'

In [520]:
df.public_meeting = pd.factorize(df.public_meeting)[0]

In [468]:
df['best_wards'] = ((df.ward == 'igosi') | (df.ward == 'imalinyi') | (df.ward == 'siha kati'))
df['second_best_wards'] = ((df.ward == 'mdandu') | (df.ward == 'nduruma'))

In [467]:
df['second_best_wards'] = ((train.ward == 'mdandu') | (train.ward == 'nduruma'))

In [461]:
train.ward.value_counts()

igosi               307
imalinyi            252
siha kati           232
mdandu              231
nduruma             217
kitunda             203
mishamo             203
msindo              201
chalinze            196
maji ya chai        190
usuka               187
ngarenanyuki        172
chanika             171
vikindu             162
mtwango             153
matola              145
zinga/ikerege       141
wanging'ombe        139
maramba             139
itete               137
magomeni            135
ifakara             134
kikatiti            134
olkokola            133
maposeni            130
igongolo            129
mvomero             129
mlangali            125
nkoma               122
nkungulu            121
                   ... 
uwanja wa ndege       1
burungura             1
linda                 1
thawi                 1
mkumbi                1
machinjioni           1
ukata                 1
korongoni             1
themi                 1
kirongo               1
kihangimahuka   

In [508]:
train[train.num_private >= 3].functional.mean()

0.6157337367624811

In [500]:
train['functional'] = (train.status_group == 'functional')

In [638]:
train.status_group.value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [514]:
train[train.num_private_over100].functional.mean()

0.66

In [515]:
train.num_private.describe()

count    59400.000000
mean         0.474141
std         12.236230
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       1776.000000
Name: num_private, dtype: float64

In [533]:
train.mac_wards

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        1
12        0
13        0
14        2
15        0
16        3
17        4
18        0
19        0
20        5
21        0
22        4
23        0
24        0
25        6
26        0
27        0
28        0
29        7
         ..
59370     0
59371     0
59372     0
59373     0
59374     0
59375     0
59376     0
59377     0
59378    15
59379     0
59380     0
59381    71
59382    44
59383     0
59384     0
59385     0
59386     0
59387     0
59388     0
59389     0
59390     0
59391     0
59392     0
59393     0
59394    64
59395    76
59396     0
59397    55
59398     0
59399     0
Name: mac_wards, Length: 59400, dtype: int64

In [560]:
df.population.describe()

count    73758.000000
mean       181.300944
std        472.383194
min          0.000000
25%          0.000000
50%         25.000000
75%        217.000000
max      30500.000000
Name: population, dtype: float64

In [589]:
df[df.amount_tsh > 25].shape

(18035, 237)

In [598]:
df['amt_tsh_pop_relationship'] = StandardScaler().fit_transform(df[['amount_tsh', 'population']]).T[0] /StandardScaler().fit_transform(df[['amount_tsh', 'population']]).T[1]

array([-0.15305674,  0.14543177, -0.38168624, ..., -0.38168624,
       -0.12977041, -0.38380318])